[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/weaviate/recipes/blob/main/integrations/llm-agent-frameworks/mem0/quickstart_mem0_with_weaviate.ipynb)

# How to integrate Long-Term Memory for AI Agents with Mem0 and Weaviate

[Mem0](https://mem0.ai/) is an open-source tools for adding a memory layer to agentic AI applications. Mem0 is designed to extract, consolidate, and retrieve information from and for on-going conversations.

This tutorial shows you how you can add long-term memory to your AI agents using Mem0 and [Weaviate](https://weaviate.io/). We will cover basic operations such as adding, searching, updating, and deleting memories.

## Step 1: Download required libraries

First, we need to install the required `mem0ai` (v0.1.118) and `weaviate-client` (v4.17.0) libraries. 

In [1]:
%%capture
%pip install -U mem0ai weaviate-client

## Step 2: Setup API Key

Mem0 uses OpenAI as the provider for the LLM and embedding model by default. Therefore, you need to set the [API key](https://platform.openai.com/docs/quickstart) `OPENAI_API_KEY` as an environment variable.

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "sk-..."

## Step 3: Start up a Weaviate vector database instance

Because, we want to use the Weaviate vector database to store the memories, we need to start up a Weaviate vector database instance.

You can choose one of the following options:

1. **Option 1:** You can create a 14-day free sandbox on the managed service [Weaviate Cloud (WCD)](https://console.weaviate.cloud/)
3. **Option 2:** You can start up a [local Weaviate vector database instance with Doker](https://docs.weaviate.io/deploy/installation-guides/docker-installation)

For this tutorial, we will start up a local Weaviate vector database instance with the following Docker command:


In [3]:
#!docker run -p 8080:8080 -p 50051:50051 cr.weaviate.io/semitechnologies/weaviate:1.33.0

## Step 4: Prepare a collection

Now, we need to connect to our vector database cluster and prepare an empty collection to hold our memories with the following properties.

In [ ]:
import weaviate
import weaviate.classes.config as wvc

client = weaviate.connect_to_local()

collection_name = "LongtermMemories"

properties = [
    wvc.Property(name="ids", data_type=wvc.DataType.TEXT),
    wvc.Property(name="hash", data_type=wvc.DataType.TEXT),
    wvc.Property(
        name="metadata",
        data_type=wvc.DataType.TEXT,
        description="Additional metadata",
    ),
    wvc.Property(name="data", data_type=wvc.DataType.TEXT),
    wvc.Property(name="created_at", data_type=wvc.DataType.TEXT),
    wvc.Property(name="category", data_type=wvc.DataType.TEXT),
    wvc.Property(name="updated_at", data_type=wvc.DataType.TEXT),
    wvc.Property(name="user_id", data_type=wvc.DataType.TEXT),
    wvc.Property(name="agent_id", data_type=wvc.DataType.TEXT),
    wvc.Property(name="run_id", data_type=wvc.DataType.TEXT),
]

# Only uncomment this if you want to delete the collection and start fresh
#if client.collections.exists(collection_name):
#    client.collections.delete(collection_name)

client.collections.create(
    name=collection_name,
    properties=properties,
)

client.close() 

## Step 5: Configure Weaviate as your vector store
Next, we can configure Mem0 to use Weaviate as the vector store.

In [ ]:
from mem0 import Memory

config = {
    "vector_store": {
        "provider": "weaviate",
        "config": {
            "collection_name": collection_name,
            "cluster_url": "http://localhost:8080",
            "auth_client_secret": None,
        }
    }
}

m = Memory.from_config(config)

## Step 6: Basic Operations of Managing Memories

Now, you're all set up and can start managing memories with the following basic operations:

### Add Memories

The `add` function extracts memories from the conversation and stores them in Weaviate for a specific user.

In [6]:
messages = [
    {"role": "user", "content": "Hi, I'm Alex. I'm a vegetarian and allergic to nuts."},
    {"role": "assistant", "content": "Hello Alex! I'll remember your dietary preferences."}
]

result = m.add(
    messages=messages,
    user_id="alex",
    metadata={"category": "user information"},
)

result

{'results': [{'id': 'c906db6c-c5af-4a2e-9aa0-d0ab7cde94f5',
   'memory': 'Name is Alex',
   'event': 'ADD'},
  {'id': 'e107bda6-887b-42b8-9a4e-cf9d2e02bec9',
   'memory': 'Is a vegetarian',
   'event': 'ADD'},
  {'id': 'ebb43ced-eb94-4e58-9739-5b688d2f9c70',
   'memory': 'Is allergic to nuts',
   'event': 'ADD'}]}

As you can see, based on the conversation "Hi, I'm Alex. I'm a vegetarian and allergic to nuts.", mem0 has extracted three memories:
- The user's name
- The user's dietary preferences
- The user's allergies

### Search Memories
Next, you can use the `search` function to look for the most related memory for a `query` and `user_id`. The default distance metric for vector search is `cosine`.

In [7]:
results = m.search(
    query="What should I cook for dinner?",
    user_id="alex",
)

results

{'results': [{'id': 'e107bda6-887b-42b8-9a4e-cf9d2e02bec9',
   'memory': 'Is a vegetarian',
   'hash': 'ce6b1c84586772ab9995a9477032df99',
   'metadata': {'category': 'user information'},
   'score': 1.0,
   'created_at': '2025-10-09T05:38:39.698896-07:00',
   'updated_at': None,
   'user_id': 'alex'},
  {'id': 'ebb43ced-eb94-4e58-9739-5b688d2f9c70',
   'memory': 'Is allergic to nuts',
   'hash': '7873cd0e5a29c513253d9fad038e758b',
   'metadata': {'category': 'user information'},
   'score': 1.0,
   'created_at': '2025-10-09T05:38:40.754045-07:00',
   'updated_at': None,
   'user_id': 'alex'},
  {'id': 'c906db6c-c5af-4a2e-9aa0-d0ab7cde94f5',
   'memory': 'Name is Alex',
   'hash': 'd0fccc8fa47f7a149ee95750c37bb0ca',
   'metadata': {'category': 'user information'},
   'score': 1.0,
   'created_at': '2025-10-09T05:38:38.645280-07:00',
   'updated_at': None,
   'user_id': 'alex'}]}

### Update Memories

You can also `update` existing memories via their id.

In [9]:
memory_id = 'e107bda6-887b-42b8-9a4e-cf9d2e02bec9'

m.update(
    memory_id=memory_id, 
    data="Is a vegan."
    )

{'message': 'Memory updated successfully!'}

Once you've updated a memory, you can also view their history via the `history` function.

In [10]:
m.history(memory_id=memory_id)

[{'id': '55481258-67aa-4680-8e90-17bc7cd74a01',
  'memory_id': 'e107bda6-887b-42b8-9a4e-cf9d2e02bec9',
  'old_memory': None,
  'new_memory': 'Is a vegetarian',
  'event': 'ADD',
  'created_at': '2025-10-09T05:38:39.698896-07:00',
  'updated_at': None,
  'is_deleted': False,
  'actor_id': None,
  'role': None},
 {'id': 'b2a02dcc-6967-4e5a-8803-f71f18ff2f38',
  'memory_id': 'e107bda6-887b-42b8-9a4e-cf9d2e02bec9',
  'old_memory': 'Is a vegetarian',
  'new_memory': 'Is a vegan.',
  'event': 'UPDATE',
  'created_at': '2025-10-09T05:38:39.698896-07:00',
  'updated_at': '2025-10-09T05:38:58.892620-07:00',
  'is_deleted': False,
  'actor_id': None,
  'role': None}]

### Delete Memories

Finally, you can also `delete` a memory.

In [11]:
memory_id = 'c906db6c-c5af-4a2e-9aa0-d0ab7cde94f5'
results = m.delete(memory_id=memory_id)

results

{'message': 'Memory deleted successfully!'}

Now, you can use the `get_all` function to view all available memories by `user_id` in Weaviate.

In [12]:
results = m.get_all(user_id="alex")

results

{'results': [{'id': 'ebb43ced-eb94-4e58-9739-5b688d2f9c70',
   'memory': 'Is allergic to nuts',
   'hash': '7873cd0e5a29c513253d9fad038e758b',
   'metadata': {'category': 'user information'},
   'created_at': '2025-10-09T05:38:40.754045-07:00',
   'updated_at': None,
   'user_id': 'alex',
   'agent_id': None,
   'run_id': None},
  {'id': 'e107bda6-887b-42b8-9a4e-cf9d2e02bec9',
   'memory': 'Is a vegan.',
   'hash': 'a8622f068d1d6721ae04930ba3416d48',
   'metadata': {'category': 'user information'},
   'created_at': '2025-10-09T05:38:39.698896-07:00',
   'updated_at': '2025-10-09T05:38:58.892620-07:00',
   'user_id': 'alex',
   'agent_id': None,
   'run_id': None}]}